# Example Algorithms

This notebook shows how to create and save a graph projection, as well as call and run specific algorithms.

In [1]:
from py2neo import Graph
import pandas as pd

### connect to the graph

In [6]:
uri = 'bolt://localhost:7687'
myGraph=Graph(uri, auth=("neo4j","pass"))

### Create a graph projection

Because many of these algorithms require monopartite graphs, I start by creating a graph projection that I can call to run my algorithms on. This saves me from creating multiple projections and also makes my algo calls more flexible - if I want to run a different projection, I can create one and swap out the projection names.

For this example, I created the most general one-hop business to business projection possible: any businesses sharing a relationship via an export or  import are linked in this projection

#### node-node  via export, import, spend

In [56]:
query = '''
CALL algo.graph.load('my-graph',
    'MATCH (d:Node) return id(d) as id',
    'MATCH (d:Node)-[r]->(e)<-[r2]-(d2:Node) WHERE e:Export OR e:Import AND d <> d2 RETURN id(d2) as source, id(d) as target, count(distinct e.id) as weight ', 
    {graph:'cypher'}
    )
'''

myGraph.run(query);

### Degree centrality

I'm going to demonstrate degree centrality on two graphs: a simple algorithm call looking at incoming and outgoing relationships from labeled nodes, as well as a version using the graph projection we created above.

#### Simple algorithm calls based on existing relationships


In [43]:
# degree centrality the graph based on incoming relationships
query='''
    CALL algo.degree.stream("LABEL",null,{direction:"incoming"})
    YIELD nodeId, score
    RETURN algo.asNode(nodeId).id as nodeID, score
    ORDER BY score DESC
'''
result=myGraph.run(query).to_data_frame()
result.head()

,DUNS,score
0,9122532,9.0
1,30472497,1.0
2,30472507,1.0
3,30472508,1.0
4,30472510,1.0


In [42]:
# degree centrality DUNS in the graph based on outgoing relationships
query='''
    CALL algo.degree.stream("LABEL",null,{direction:"outgoing"})
    YIELD nodeId, score
    RETURN algo.asNode(nodeId).id as nodeID, score
    ORDER BY score DESC
'''
result=myGraph.run(query).to_data_frame()
result.head()

,DUNS,score
0,9122532,9.0
1,30472497,1.0
2,30472507,1.0
3,30472508,1.0
4,30472510,1.0


In [47]:
# degree centrality in the graph based on relationships regardless of direction
query='''
    CALL algo.degree.stream("LABEL",null,{direction:"both"})
    YIELD nodeId, score
    RETURN algo.asNode(nodeId).id as nodeID, score
    ORDER BY score DESC
'''
result=myGraph.run(query).to_data_frame()
result.head()

,DUNS,score
0,9122532,9.0
1,30472497,1.0
2,30472507,1.0
3,30472508,1.0
4,30472510,1.0


#### Degree Centrality based on the graph projection, created above.

In [57]:
# degree centrality in the graph based on outgoing relationships
query='''
    CALL algo.degree.stream(null,null,{graph:"my-graph",direction:"outgoing"})
    YIELD nodeId, score
    RETURN algo.asNode(nodeId).id as nodeID, score
    ORDER BY score DESC
'''
result=myGraph.run(query).to_data_frame()
result.head()

,DUNS,score
0,0,93.0
1,752524058,16.0
2,51955774,16.0
3,840474894,12.0
4,964698521,12.0


In [46]:
# degree centrality in the graph based on incoming relationships
query='''
    CALL algo.degree.stream(null,null,{graph:"my-graph",direction:"incoming"})
    YIELD nodeId, score
    RETURN algo.asNode(nodeId).id as nodeID, score
    ORDER BY score DESC
'''
result=myGraph.run(query).to_data_frame()
result.head()

,DUNS,score
0,30472497,0.0
1,30472507,0.0
2,30472508,0.0
3,30472510,0.0
4,30472521,0.0


Given the directionality assigned in our graph projection, this makes sense - all the relationships are outgoing (note the order of source and target in the graph.load function).

In [48]:
# degree centrality in the graph based on either direction of relationships
query='''
    CALL algo.degree.stream(null,null,{graph:"my-graph",direction:"both"})
    YIELD nodeId, score
    RETURN algo.asNode(nodeId).id as nodeID, score
    ORDER BY score DESC
'''
result=myGraph.run(query).to_data_frame()
result.head()

,DUNS,score
0,0,93.0
1,752524058,16.0
2,51955774,16.0
3,840474894,12.0
4,964698521,12.0


### Betweenness Centrality

As above, I'll demonstrate the algorithm call for labeled nodes as well as the graph projection.

In [50]:
# degree centrality in the graph based on incoming relationships
query='''
    CALL algo.betweenness.stream("LABEL",null,{direction:"in"})
    YIELD nodeId, centrality
    RETURN algo.asNode(nodeId).id as nodeID, centrality
    ORDER BY centrality DESC
'''
result=myGraph.run(query).to_data_frame()
result.head()

,DUNS,centrality
0,9122532,72.0
1,30472497,0.0
2,30472507,0.0
3,30472508,0.0
4,30472510,0.0


In [51]:
# degree centrality in the graph based on outgoing relationships
query='''
    CALL algo.betweenness.stream("LABEL",null,{direction:"out"})
    YIELD nodeId, centrality
    RETURN algo.asNode(nodeId).id as id, centrality
    ORDER BY centrality DESC
'''
result=myGraph.run(query).to_data_frame()
result.head()

,DUNS,centrality
0,9122532,72.0
1,30472497,0.0
2,30472507,0.0
3,30472508,0.0
4,30472510,0.0


In [52]:
# degree centrality in the graph based on undirected relationships
query='''
    CALL algo.betweenness.stream("LABEL",null,{direction:"both"})
    YIELD nodeId, centrality
    RETURN algo.asNode(nodeId).id as nodeID, centrality
    ORDER BY centrality DESC
'''
result=myGraph.run(query).to_data_frame()
result.head()

,DUNS,centrality
0,9122532,36.0
1,30472497,0.0
2,30472507,0.0
3,30472508,0.0
4,30472510,0.0


#### Graph Projection

In [53]:
# degree centrality in the graph based on undirected relationships
query='''
    CALL algo.betweenness.stream(null,null,{graph:'my-graph',direction:"in"})
    YIELD nodeId, centrality
    RETURN algo.asNode(nodeId).id as nodeID, centrality
    ORDER BY centrality DESC
'''
result=myGraph.run(query).to_data_frame()
result.head()

,DUNS,centrality
0,30472497,0.0
1,30472507,0.0
2,30472508,0.0
3,30472510,0.0
4,30472521,0.0


In [54]:
# degree centrality the graph based on undirected relationships
query='''
    CALL algo.betweenness.stream(null,null,{graph:'my-graph',direction:"out"})
    YIELD nodeId, centrality
    RETURN algo.asNode(nodeId).id as id, centrality
    ORDER BY centrality DESC
'''
result=myGraph.run(query).to_data_frame()
result.head()

,DUNS,centrality
0,0,894.0
1,752524058,8.0
2,30472497,0.0
3,30472507,0.0
4,30472508,0.0


In [55]:
# degree centrality in the graph based on undirected relationships
query='''
    CALL algo.betweenness.stream(null,null,{graph:'my-graph',direction:"both"})
    YIELD nodeId, centrality
    RETURN algo.asNode(nodeId).id as nodeID, centrality
    ORDER BY centrality DESC
'''
result=myGraph.run(query).to_data_frame()
result.head()

,DUNS,centrality
0,0,447.0
1,752524058,4.0
2,30472497,0.0
3,30472507,0.0
4,30472508,0.0


### Pagerank
For PageRank, we need to use a monopartite graph - I'm going to demonstrate PageRank just using the graph I created above. The weight parameter is based on the number of shared entities between any two labeled nodes, but you could swap it out in the graph projection if something else seemed more intuitive.

In [59]:
query='''
    CALL algo.pageRank.stream(null,null,{graph:'my-graph',weightProperty:'weight', iterations:10, dampingFactor:0.85, direction:"both"})
    YIELD nodeId, score
    RETURN algo.asNode(nodeId).id as id, score
    ORDER BY score DESC
'''
result=myGraph.run(query).to_data_frame()
result.head()

,DUNS,score
0,0,8.249140
1,752524058,1.541602
2,662747620,1.190544
3,51955774,1.113180
4,840474894,1.081181


### Louvain Clustering

Louvain also requires a monopartite graph. I'll use the same graph projection here, including weights, as I used for page rank, but consider playing around with the weighting and projection to create different communities.

In [2]:
query='''
    CALL algo.louvain.stream(null,null,{graph:'my-graph',weightProperty:'weight', iterations:10, dampingFactor:0.85, direction:"both"})
    YIELD nodeId, community
    RETURN algo.asNode(nodeId).id AS nodeID, community
    ORDER BY community;
'''
result=myGraph.run(query).to_data_frame()
#result.head()

### Triangle Counting

We'll continue using the projected graph to run triangle counting.

In [63]:
query='''
    CALL algo.triangleCount.stream(null,null,{graph:'my-graph'})
    YIELD nodeId, triangles, coefficient
    RETURN algo.asNode(nodeId).id AS nodeID, triangles, coefficient
    ORDER BY coefficient DESC

'''
result=myGraph.run(query).to_data_frame()
result.head()

,DUNS,coefficient,triangles
0,78515699,1.333333,8
1,8714550,1.333333,8
2,527210056,1.000000,3
3,663152692,0.833333,5
4,24926387,0.833333,5


### Removing the graph projection

When you've finished your algorithm calls, you want to delete your graph projection to free up memory.

In [65]:
query='''
CALL algo.graph.remove('my-graph')
YIELD name, type, exists, removed, nodes;
'''
myGraph.run(query);